# Description

Script to run the agent on the hardware

0. Description
1. Imports
2. Paths
3. Configuration
4. Functions
5. Creation
6. Run on Hardware
7. Save recorded Data


# Specification

In [1]:
take = 108
speed = 100
dt = 0.12
max_limit = 0.00012
info = "ptc"

# Imports

In [2]:
# General imports
import os
import time
import datetime
from os.path import expanduser
import pandas
import pickle
import numpy
from matplotlib import pyplot as plt

# rllib imports
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune.registry import get_trainable_cls
from ray.tune.registry import register_env
from ray.rllib.env import MultiAgentEnv
from ray.rllib.env.base_env import _DUMMY_AGENT_ID
from ray.rllib.evaluation.worker_set import WorkerSet
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.utils.deprecation import deprecation_warning

# Gym imports
from gym.envs.robotics.ur10 import randomize
from gym.envs.robotics import ur10_noisy_pd_env
from gym import utils
from gym.envs.robotics import rotations, robot_custom_env, utils

# MuJoCo imports
import mujoco_py

# ROS imports
import rospy
from sensor_msgs.msg import JointState
from std_msgs.msg import Float64MultiArray
from std_msgs.msg import String
from geometry_msgs.msg import WrenchStamped
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from realsense2_camera.msg import CustomVector
from realsense2_camera.msg import EstimatedPose

/home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

# Paths

In [3]:
HOME_PATH = os.getenv("HOME")
PROJECT_PATH = os.path.join(*[HOME_PATH, "DRL_AI4RoMoCo"])
SAVE_PATH = os.path.join(*[PROJECT_PATH, "code", "data"])
XML_MODEL_PATH = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation","UR10","ur10_heg.xml"])

# Configuration

Hardware - Environment (ROS)

In [4]:
actual_q = numpy.zeros(6,)
ft_values = numpy.zeros(6,)

Simulation - Environment (MuJoCo, Gym)

In [5]:
goal = numpy.array([0.69423743, -0.83110109,  1.17388998, -1.57161506,  0.02185773, -3.14102438])
reward_type = "dense"
distance_threshold = 0.002
fail_threshold = 0.25
punish_force = False
punish_force_thresh = 20
punish_force_factor = 0.001
print(goal)

[ 0.69423743 -0.83110109  1.17388998 -1.57161506  0.02185773 -3.14102438]


In [6]:
initial_qpos_simpheg_conf2 = numpy.array([0, -1.3, 2.1, -0.80, 1.5708, 0])
randomize_kwargs = {
    'var_mass':     0,#0.5,
    'var_damp':     0,#0.5,
    'var_fr':       0,#0.5,
    'var_grav_x_y': 0,#0.5,
    'var_grav_z':   0,#1,
    'var_body_pos': 0,#0.05,
    'var_body_rot': 0,
}

Agent (ray, tune, rllib)

In [7]:
checkpoint_model1 = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model1", "checkpoint-2120"])
checkpoint_model2 = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model2", "checkpoint-3440"])
checkpoint_model3 = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model3", "checkpoint-3030"])
checkpoint_model4 = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model4", "checkpoint-3590"])
checkpoint_model5 = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model5", "checkpoint-4410"])
checkpoint = checkpoint_model4
checkpoint

'/home/lars/DRL_SetBot-RearVentilation/final_models/model4/checkpoint-3590'

In [8]:
config = {}
    # Load configuration from checkpoint file.
config_dir = os.path.dirname(checkpoint)
config_path = os.path.join(config_dir, "params.pkl")
# Try parent directory.
if not os.path.exists(config_path):
    config_path = os.path.join(config_dir, "../params.pkl")
if os.path.exists(config_path):
    print("Config found!")
    with open(config_path, "rb") as f:
            config = pickle.load(f)
            
config['num_workers'] = 2
config['num_gpus'] = 1

Config found!


In [9]:
config

{'env': 'RandomizedEnv',
 'callbacks': {'on_episode_start': <function __main__.on_episode_start(info)>,
  'on_episode_step': <function __main__.on_episode_step(info)>,
  'on_episode_end': <function __main__.on_episode_end(info)>},
 'num_workers': 2,
 'num_gpus': 1,
 'num_cpus_per_worker': 1,
 'horizon': 2500,
 'lr': 0.0003,
 'train_batch_size': 57500,
 'model': {'use_lstm': False,
  'lstm_cell_size': 128,
  'fcnet_hiddens': [128, 128]},
 'tf_session_args': {'allow_soft_placement': True}}

# Functions

Helper Functions

In [10]:
def normalize_rad(angles):
    angles = numpy.array(angles)
    angles = angles % (2*numpy.pi)
    angles = (angles + 2*numpy.pi) % (2*numpy.pi)
    for i in range(len(angles)):
        if (angles[i] > numpy.pi):
            angles[i] -= 2*numpy.pi
    return angles

def get_obs():
    set_state(actual_q)
    for i in range(10):
        viewer.render()
    #print(ft_values)
    x_pos = sim.data.get_body_xpos("gripper_dummy_heg")
    x_mat = sim.data.get_body_xmat("gripper_dummy_heg")
    rpy =  normalize_rad(rotations.mat2euler(x_mat))
    #print(x_pos, rpy)

    obs = numpy.concatenate([
                x_mat.dot(x_pos-goal[:3]), x_mat.dot(normalize_rad(rpy-goal[3:])), ft_values.copy()
            ])
    return obs

def get_dq(dx):
    jacp = sim.data.get_body_jacp(name="gripper_dummy_heg").reshape(3, 6)
    jacr = sim.data.get_body_jacr(name="gripper_dummy_heg").reshape(3, 6)
    jac = numpy.vstack((jacp, jacr))
    dq = numpy.linalg.lstsq(jac, dx)[0].reshape(6, )
    return dq

def go_smooth(q):
    rate = rospy.Rate(125)
    dq = q - actual_q
    while numpy.linalg.norm(dq)>0.005:
        dq = q - actual_q
        dq_ = numpy.clip(dq, -0.0001, 0.0001)
        control(ctrl_q+dq_)
        rate.sleep()
    control(q)
    
def set_state(qpos):
    #assert qpos.shape == (model.nq,)
    old_state = sim.get_state()
    new_state = mujoco_py.MjSimState(old_state.time, qpos, old_state.qvel,
                                     old_state.act, old_state.udd_state)
    sim.set_state(new_state)
    sim.forward()
    
def keep_going(steps, num_steps, episodes, num_episodes):
    """Determine whether we've collected enough data"""
    # if num_episodes is set, this overrides num_steps
    if num_episodes:
        return episodes < num_episodes
    # if num_steps is set, continue until we reach the limit
    if num_steps:
        return steps < num_steps
    # otherwise keep going forever
    return True

ROS

In [11]:
# ROS callback functions
def q_callback(data):
    '''Callback for the joint angles, velocities and accelerations'''
    global actual_q
    global actual_qd
    global actual_qdd
    actual_q = numpy.array(data.position)[[2,1,0,3,4,5]]
    actual_qd = numpy.array(data.velocity)[[2,1,0,3,4,5]]
    actual_qdd = numpy.array(data.effort)[[2,1,0,3,4,5]]
    set_state(actual_q)
    viewer.render()
    
def ft_callback(data):
    '''Callback for the Robotiq FT300 force torque values'''
    global ft_values
    ft_values = 5 * numpy.array([data.wrench.force.x, data.wrench.force.y, data.wrench.force.z, data.wrench.torque.x, \
                            data.wrench.torque.y, data.wrench.torque.z])
    
def pose_callback(data):
    '''Callback for the Intel Realsense D435i and OpenCV charuco pose estimation'''
    global pose
    pose = numpy.array([data.tx, data.ty, data.tz, data.rx, data.ry, data.rz])
    
def observation_callback(data):
    '''Callback for the entire observation array (same as in simulation)'''
    global observations
    observations = numpy.array(data.data)
    
def check_ROS_topics():
    '''Listens to all required ROS topics and checks if each of them is online and sending'''
    check_jointstates = []
    check_ft300_force_torque = []
    check_observations = []
    check_pose_estimation = []
    
    for i in range(5):
        check_jointstates.append(actual_q)
        check_ft300_force_torque.append(ft_values)
        check_observations.append(observations)
        check_pose_estimation.append(pose)
        time.sleep(0.5)
        
    topics = ["/joint_states", "/ft300_force_torque", "/observation", "/pose_estimation"]
    topics_dict = {
        "/joint_states" : check_jointstates,
        "/ft300_force_torque" : check_ft300_force_torque,
        "/observation" : check_observations,
        "/pose_estimation" : check_pose_estimation
    }
    
    for topic in topics:
        if not topics_dict[topic]:
            print("ERROR: {} is offline or not sending".format(topic))
        else:
            print("SUCCESS: {} is online and sending".format(topic))

In [12]:
def goal_distance(obs, goal):
    '''Compute distance between TCP and Goal'''
    obs = obs[:6]
    assert obs.shape == goal.shape
    return numpy.linalg.norm(obs*numpy.array([1, 1, 1, 0.3, 0.3, 0.3]), axis=-1)

def compute_reward(obs, goal):
    '''Compute distance between goal and the achieved goal.'''
    d = goal_distance(obs, goal)
    if reward_type == 'sparse':
        return -(d > distance_threshold).astype(numpy.float32) - 10*(d > fail_threshold).astype(numpy.float)
    else:
        rew = -d
        force_amp = numpy.linalg.norm(obs[6:9])
        if punish_force and force_amp > punish_force_thresh:
            rew -= punish_force_factor * force_amp
        return rew

In [13]:
# ROS control commands
def control(q):
    '''Controlling the Universal Robot UR10 JointGroupPositionController'''
    global ctrl_q
    command = Float64MultiArray()
    command.data = q
    control_publisher.publish(command)
    ctrl_q = q.copy()
    
def traj_control(q):
    #dt = 0.025
    global ctrl_q
    command = JointTrajectory()
    point = JointTrajectoryPoint()
    
    #command.header.stamp = rospy.Time.now()
    #command.header.frame_id = "DRL_UR10"
    #command.joint_names = ["elbow_joint", "shoulder_lift_joint", "shoulder_pan_joint", "wrist_1_joint", "wrist_2_joint", "wrist_3_joint"]
    
    #command.header.stamp = rospy.Time.now()
    command.header.frame_id = "DRL_UR10"
    command.joint_names.append("shoulder_pan_joint")
    command.joint_names.append("shoulder_lift_joint")
    command.joint_names.append("elbow_joint")
    command.joint_names.append("wrist_1_joint")
    command.joint_names.append("wrist_2_joint")
    command.joint_names.append("wrist_3_joint")
    

    point.positions.append(q[0])
    point.positions.append(q[1])
    point.positions.append(q[2])
    point.positions.append(q[3])
    point.positions.append(q[4])
    point.positions.append(q[5])
    
    command.points.append(point)
    command.points[0].time_from_start=rospy.Duration.from_sec(dt)
    
    traj_control_publisher.publish(command)
    ctrl_q = q.copy()
    
def static_approach():
    
    waypoint0 = numpy.array([1.6256170272827148, -0.9155753294574183, 1.5307668447494507, -0.6977546850787562, 1.5475375652313232, 0.011224725283682346])[[2,1,0,3,4,5]]
    waypoint1 = numpy.array([2.47381329536438, -1.4324410597430628, 1.5743204355239868, -1.0292657057391565, 1.5910834074020386, 0.011859889142215252])[[2,1,0,3,4,5]]
    waypoint2 = numpy.array([2.519468069076538, -1.6841238180743616, 0.13313426077365875, -0.8226664702044886, 1.7688652276992798, 0.014099815860390663])[[2,1,0,3,4,5]]
    waypoint3 = numpy.array([2.1001405715942383, -1.3000262419330042, 7.190534961409867e-05, -0.7999804655658167, 1.5709056854248047, -3.5587941304981996e-05])[[2,1,0,3,4,5]]
    
    dt_static = 2
    command = JointTrajectory()
    point0 = JointTrajectoryPoint()
    point1 = JointTrajectoryPoint()
    point2 = JointTrajectoryPoint()
    point3 = JointTrajectoryPoint()
    
    command.header.stamp = rospy.Time.now()
    command.header.frame_id = "DRL_UR10"
    command.joint_names.append("shoulder_pan_joint")
    command.joint_names.append("shoulder_lift_joint")
    command.joint_names.append("elbow_joint")
    command.joint_names.append("wrist_1_joint")
    command.joint_names.append("wrist_2_joint")
    command.joint_names.append("wrist_3_joint")
    
    point0.positions.append(waypoint0[0])
    point0.positions.append(waypoint0[1])
    point0.positions.append(waypoint0[2])
    point0.positions.append(waypoint0[3])
    point0.positions.append(waypoint0[4])
    point0.positions.append(waypoint0[5])
    
    #command.points.append(point0)
    
    point1.positions.append(waypoint1[0])
    point1.positions.append(waypoint1[1])
    point1.positions.append(waypoint1[2])
    point1.positions.append(waypoint1[3])
    point1.positions.append(waypoint1[4])
    point1.positions.append(waypoint1[5])
    
    command.points.append(point1)
    command.points[0].time_from_start=rospy.Duration.from_sec(dt_static)
    
    point2.positions.append(waypoint2[0])
    point2.positions.append(waypoint2[1])
    point2.positions.append(waypoint2[2])
    point2.positions.append(waypoint2[3])
    point2.positions.append(waypoint2[4])
    point2.positions.append(waypoint2[5])
    
    command.points.append(point2)
    command.points[1].time_from_start=rospy.Duration.from_sec(2*dt_static)
    
    point3.positions.append(waypoint3[0])
    point3.positions.append(waypoint3[1])
    point3.positions.append(waypoint3[2])
    point3.positions.append(waypoint3[3])
    point3.positions.append(waypoint3[4])
    point3.positions.append(waypoint3[5])
    
    command.points.append(point3)
    command.points[2].time_from_start=rospy.Duration.from_sec(2.5*dt_static)
    
    traj_control_publisher.publish(command)
    
def ft_zero():
    '''Zeroes the Robotiq FT300 force torque sensor by sending an ur_script via UR Secondary Interface'''
    zero_sensor_urscript = 'sec ft_set_zero():' + '\n' + '\t' + 'if(socket_open("127.0.0.1", 63350, "acc")):' \
          + '\n' + '\t' + '\t' + 'socket_send_string("SET ZRO", "acc")' \
         + '\n' + '\t' + '\t' + 'socket_close("acc")' + '\n' + '\t' + 'end' + '\n' + 'end' + '\n'
    ft_zero_publisher.publish(zero_sensor_urscript)

# Creation

Simulation Environment (Gym)

In [14]:
model = mujoco_py.load_model_from_path(XML_MODEL_PATH)
sim = mujoco_py.MjSim(model)
q_init = numpy.array([0, -1.3, 2.1, -0.80, 1.5708, 0.0])

ctrl_q = q_init
set_state(q_init)

viewer = mujoco_py.MjViewer(sim)
viewer.render()

Creating window glfw


In [15]:
x_pos = sim.data.get_body_xpos("gripper_dummy_heg")
x_mat = sim.data.get_body_xmat("gripper_dummy_heg")
rpy =  normalize_rad(rotations.mat2euler(x_mat))
print(x_pos, rpy)

[ 0.57243453 -0.83605904  1.17575472] [-1.57079633e+00  3.67320510e-06  3.14159265e+00]


In [16]:
class RandomizedEnv(ur10_noisy_pd_env.Ur10Env):
    def __init__(self, env_config):
        model_xml_path = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "UR10", "ur10_assembly_setup_rand_temp_{}.xml".format(env_config.worker_index)])
        randomize.randomize_ur10_xml(worker_id = env_config.worker_index)
        ur10_noisy_pd_env.Ur10Env.__init__(
            self, model_xml_path, n_substeps=80, distance_threshold=0.002, initial_qpos=initial_qpos_simpheg_conf2,
            reward_type='dense', ctrl_type='cartesian', vary=True, worker_id=env_config.worker_index,
            randomize_kwargs=randomize_kwargs)
        #utils.EzPickle.__init__(self)

register_env("RandomizedEnv", lambda config: RandomizedEnv(config))

Agent

In [17]:
ray.init()
cls = get_trainable_cls("PPO")

2020-07-23 16:53:03,487	INFO resource_spec.py:212 -- Starting Ray with 15.19 GiB memory available for workers and up to 7.62 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-07-23 16:53:04,090	INFO services.py:1083 -- View the Ray dashboard at localhost:8265


In [18]:
#agent = cls(env="RandomizedEnv", config=config)
agent = cls(config=config)
agent.restore(checkpoint)

2020-07-23 16:53:04,808	INFO trainer.py:420 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-07-23 16:53:04,886	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/home/lars/DRL_SetBot-RearVentilation/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=15240) /home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=15240)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=15240) /home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=15240)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=15240) /home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=15240)   _np_qint16 = np.dtype([("qint16", np.int16, 1)])
(pid=15240) /home/lars/.l

2020-07-23 16:53:11,274	WARNING trainable.py:210 -- Getting current IP.
2020-07-23 16:53:11,276	INFO trainable.py:416 -- Restored on 10.180.145.69 from checkpoint: /home/lars/DRL_SetBot-RearVentilation/final_models/model4/checkpoint-3590
2020-07-23 16:53:11,277	INFO trainable.py:423 -- Current state after restoring: {'_iteration': 3590, '_timesteps_total': 55108000, '_time_total': 301263.58026504517, '_episodes_total': 164361}


In [19]:
print(ray)
print(get_trainable_cls)
print(ray.tune.registry)
clss = get_trainable_cls("PPO")

<module 'ray' from '/home/lars/.local/lib/python3.6/site-packages/ray/__init__.py'>
<function get_trainable_cls at 0x7f732f6a5268>
<module 'ray.tune.registry' from '/home/lars/.local/lib/python3.6/site-packages/ray/tune/registry.py'>


ROS-nodes

In [20]:
rospy.init_node("drl_experiment", anonymous=True)
rospy.Subscriber("/joint_states", JointState, q_callback)
rospy.Subscriber("/ft300_force_torque", WrenchStamped, ft_callback)
rospy.Subscriber("/pose_estimation", EstimatedPose, pose_callback)
rospy.Subscriber("/observation", CustomVector, observation_callback)
control_publisher = rospy.Publisher("/joint_group_position_controller/command", Float64MultiArray, queue_size=1)
traj_control_publisher = rospy.Publisher("/pos_traj_controller/command", JointTrajectory, queue_size=1)
ft_zero_publisher = rospy.Publisher("/ur_hardware_interface/script_command", String, queue_size=1)

#rate = rospy.Rate(1)
rate = rospy.Rate(125)

Data structures for storing and saving data

In [21]:
observations_rec = []
poses_rec = []
ft_values_rec = []
q_rec = []
qd_rec = []
rewards = []

(pid=15242) /home/lars/DRL_SetBot-RearVentilation/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=15242)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=15240) /home/lars/DRL_SetBot-RearVentilation/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=15240)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


# Run On Hardware

In [22]:
agent.compute_action([0,0,0,0,0,0,0,0,0,0,0,0])

array([-1.        , -0.45193988, -1.        ,  0.46192476,  0.42713162,
        0.73037136], dtype=float32)

In [23]:
'''
time.sleep(2)
static_approach()
ctrl_q = actual_q
go_smooth(q_init) #go slowly to initial position
print(actual_q)
'''

'\ntime.sleep(2)\nstatic_approach()\nctrl_q = actual_q\ngo_smooth(q_init) #go slowly to initial position\nprint(actual_q)\n'

In [24]:
done = False
ft_zero()
#max_limit = 0.00075

time.sleep(4)
static_approach()
time.sleep(10)

while not done:
    #obs = get_obs()
    obs = observations
    action = agent.compute_action(obs)
    for i in range(1):
        dx = max_limit * action.copy()
        rot_mat = sim.data.get_body_xmat('gripper_dummy_heg')
        dx_ = numpy.concatenate([rot_mat.dot(dx[:3]), rot_mat.dot(dx[3:])])  ## transform to right coordinate system
        dq  = get_dq(dx_)
        q = ctrl_q + dq
        #control(q)
        traj_control(q)
        
        observations_rec.append(observations)
        poses_rec.append(pose)
        ft_values_rec.append(ft_values)
        q_rec.append(actual_q)
        qd_rec.append(actual_qd)
        rewards.append(compute_reward(obs, goal))
        
        #rate.sleep()
    rate.sleep()

/home/lars/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/home/lars/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/home/lars/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rc

/home/lars/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/home/lars/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/home/lars/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rc

KeyboardInterrupt: 

# !!! Save recorded Data !!!

In [25]:
#take = 1
model = 4
#info = "jgpc" #"ptc"
date = (datetime.datetime.now()).strftime("%y%m%d")
SAVE_PATH = os.path.join(*[PROJECT_PATH, "code", "data","TEST_Movement_Behavior"])

observations_rec_df = pandas.DataFrame(observations_rec, columns=["pos1", "pos2", "pos3", "rotx", "roty", "rotz", "Fx", "Fy", "Fz", "Mx", "My", "Mz"])
poses_rec_df = pandas.DataFrame(poses_rec, columns=["tx", "ty", "tz", "rx", "ry", "rz"])
ft_values_rec_df = pandas.DataFrame(ft_values_rec, columns=["Fx", "Fy", "Fz", "Mx", "My", "Mz"])
q_rec_df = pandas.DataFrame(q_rec, columns=["b", "s", "e", "w1", "w2", "w3"])
qd_rec_df = pandas.DataFrame(qd_rec, columns=["b", "s", "e", "w1", "w2", "w3"])
rewards_df = pandas.DataFrame(rewards, columns=["Reward"])
    
DIRECTORY = os.path.join(*[SAVE_PATH])
observations_rec_df.to_feather(os.path.join(*[DIRECTORY, "{}_obs_{}_dt_{}_maxlimit_{}_speed_{}.ftr".format(take, info, dt, max_limit, speed)]))
poses_rec_df.to_feather(os.path.join(*[DIRECTORY, "{}_poses_{}_dt_{}_maxlimit_{}_speed_{}.ftr".format(take, info, dt, max_limit, speed)]))
ft_values_rec_df.to_feather(os.path.join(*[DIRECTORY, "{}_fts_{}_dt_{}_maxlimit_{}_speed_{}.ftr".format(take, info, dt, max_limit, speed)]))
q_rec_df.to_feather(os.path.join(*[DIRECTORY, "{}_q_{}_dt_{}_maxlimit_{}_speed_{}.ftr".format(take, info, dt, max_limit, speed)]))
qd_rec_df.to_feather(os.path.join(*[DIRECTORY, "{}_qd_{}_dt_{}_maxlimit_{}_speed_{}.ftr".format(take, info, dt, max_limit, speed)]))
rewards_df.to_feather(os.path.join(*[DIRECTORY, "{}_rewards_{}_dt_{}_maxlimit_{}_speed_{}.ftr".format(take, info, dt, max_limit, speed)]))

# --- Visualize the Data with Visualize_Hardware_Tests.ipynb ---